In [4]:
import cv2
import numpy as np
import torch
import torchvision.transforms as transforms

# Load pre-trained model
model = torch.hub.load('pytorch/vision:v0.9.0', 'deeplabv3_resnet101', pretrained=True)

# Open video file
cap = cv2.VideoCapture('Girlwithacat.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize image to model input size
    input_size = (512, 512)
    resized_frame = cv2.resize(frame, input_size)

    # Normalize image
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    normalized_frame = transform(resized_frame)

    # Preprocess image for model input
    input_image = normalized_frame.unsqueeze(0)

    # Get model output
    output_image = model(input_image)['out']

    # Convert output to binary image
    binary_image = torch.argmax(output_image.squeeze(), dim=0).detach().cpu().numpy()
    binary_image = np.expand_dims(binary_image, axis=-1)

    # Resize binary image to original size
    binary_image = cv2.resize(binary_image, (frame.shape[1], frame.shape[0]))

    # Apply semantic segmentation to original image
    masked_frame = cv2.bitwise_and(frame, frame, mask=binary_image.astype(np.uint8))

    # Display result
    cv2.imshow('Semantic Segmentation', masked_frame)
    if cv2.waitKey(1) == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\Administrator/.cache\torch\hub\pytorch_vision_v0.9.0


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1266: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
